# 2024/02/14更新
- 書籍発売後にライブラリのバージョンアップが生じたため、書籍のコードが一部動作しなくなりました
- このため、書籍のコードが動作するようにコードを一部変更
    - 変更を最小化するため、基本的には書籍に合わせてライブラリをダウングレード
    - 名称変更となったライブラリは最新のライブラリ名に変更
- なお、一部ですが、2024/02/14時点の最新ライブラリでも動作するコードをコメントアウトで同じセルに参考までに掲載しました。

In [ ]:
# 引数が大きく変更されているため、ダウングレードで対応
# 最初に実行してください。
!pip install pandas==1.3.5
!pip install lightgbm==3.3.1
!pip install scikit-learn==1.0.2

# なお、LightGBMの最新版ではCallbackが使われており、過去バージョンと大きく書き方が変化。最新版を使い方を知りたい場合は公式ページを参照してください。
# https://lightgbm.readthedocs.io/en/latest/index.html

# Kaggleで磨く 機械学習の実践力
# 第4章 ベースライン作成

# 4.3 ファイルの読み込み
#### スクリプト4-1: ライブラリの読み込み

In [ ]:
import numpy as np
import pandas as pd
import os
import pickle
import gc 

# 分布確認
# import pandas_profiling as pdp
import ydata_profiling as pdp # ライブラリ名称が変更になったため

# 可視化
import matplotlib.pyplot as plt

# 前処理
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder

# バリデーション
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold

# 評価指標
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

# モデリング: lightgbm
import lightgbm as lgb

import warnings
warnings.filterwarnings("ignore")

# matplotilbで日本語表示したい場合はこれをinstallしてインポートする
!pip install japanize-matplotlib
import japanize_matplotlib
%matplotlib inline

#### スクリプト4-2: ファイルの読み込み

In [ ]:
df_train = pd.read_csv("../input/titanic/train.csv")
df_train.head()

# 4.4 データの確認（簡易）
## 4.4.1 レコード数とカラム数の確認

#### スクリプト4-3: レコード数とカラム数の確認

In [ ]:
print(df_train.shape)
print("レコード数:", len(df_train))
print("カラム数:", len(df_train.columns))

## 4.4.2 カラムごとのデータの種類の確認

#### スクリプト4-4: データの確認

In [ ]:
df_train.info()

#### スクリプト4-5: データ型の変換

In [ ]:
df_train["Pclass"] = df_train["Pclass"].astype(object)
df_train[["Pclass"]].info()

#### スクリプト4-6: データ型をobject型からint型に戻す

In [ ]:
df_train["Pclass"] = df_train["Pclass"].astype(np.int64)
df_train[["Pclass"]].info()

## 4.4.3 欠損値の確認
#### スクリプト4-7: 欠損値の確認

In [ ]:
df_train.isnull().sum()

# 4.5 データセット作成
#### スクリプト4-8: データセット作成

In [ ]:
x_train, y_train, id_train = df_train[["Pclass", "Fare"]], \
                             df_train[["Survived"]], \
                             df_train[["PassengerId"]]
print(x_train.shape, y_train.shape, id_train.shape)

# 4.6 バリデーション設計
## 4.6.2 ホールドアウト検証と交差検証
#### スクリプト4-9: ホールドアウト検証の実行

In [ ]:
x_tr, x_va, y_tr, y_va = train_test_split(x_train,
                                          y_train,
                                          test_size=0.2,
                                          shuffle=True,
                                          stratify=y_train, 
                                          random_state=123)
print(x_tr.shape, y_tr.shape)
print(x_va.shape, y_va.shape)
print("y_train:{:.3f}, y_tr:{:.3f}, y_va:{:.3f}".format(
    y_train["Survived"].mean(),
    y_tr["Survived"].mean(),
    y_va["Survived"].mean(),
))

#### スクリプト4-10: クロスバリデーションの実行

In [ ]:
n_splits = 5
cv = list(StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=123).split(x_train, y_train))
for nfold in np.arange(n_splits):
    print("-"*20, nfold, "-"*20)
    idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
    x_tr, y_tr = x_train.loc[idx_tr, :], y_train.loc[idx_tr, :]
    x_va, y_va = x_train.loc[idx_va, :], y_train.loc[idx_va, :]
    print(x_tr.shape, y_tr.shape)
    print(x_va.shape, y_va.shape)
    print("y_train:{:.3f}, y_tr:{:.3f}, y_va:{:.3f}".format(
        y_train["Survived"].mean(),
        y_tr["Survived"].mean(),
        y_va["Survived"].mean(),
    ))
    
    # ここでモデル学習（ここは次節にて説明するため省略）

# 4.7 モデル学習（勾配ブースティング）
## 4.7.1 ホールドアウト検証の場合
#### スクリプト4-11: データセットの作成（スクリプト4-9の再掲）

In [ ]:
x_tr, x_va, y_tr, y_va = train_test_split(x_train,
                                          y_train,
                                          test_size=0.2,
                                          shuffle=True,
                                          stratify=y_train, 
                                          random_state=123)
print(x_tr.shape, y_tr.shape)
print(x_va.shape, y_va.shape)
print("y_train:{:.3f}, y_tr:{:.3f}, y_va:{:.3f}".format(
    y_train["Survived"].mean(),
    y_tr["Survived"].mean(),
    y_va["Survived"].mean(),
))

#### スクリプト4-12: モデル学習（ホールドアウト検証の場合）

In [ ]:
# ハイパーパラメータ
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary', 
    'metric': 'auc',
    'learning_rate': 0.1,
    'num_leaves': 16,
    'n_estimators': 100000,
    "random_state": 123,
    "importance_type": "gain",
}

model = lgb.LGBMClassifier(**params)
model.fit(x_tr,
          y_tr,
          eval_set=[(x_tr,y_tr), (x_va,y_va)],
          early_stopping_rounds=100,
          verbose=10,
         )

# # 2024/02/14環境で動かしたい場合はこのコードを利用してください。
# model.fit(x_tr,
#           y_tr,
#           eval_set=[(x_tr,y_tr), (x_va,y_va)],
#           callbacks=[
#               lgb.early_stopping(stopping_rounds=100, verbose=True),
#               lgb.log_evaluation(10),
#           ],
#          )

#### スクリプト4-13: 精度の評価

In [ ]:
y_tr_pred = model.predict_proba(x_tr)[:,1]
y_va_pred = model.predict_proba(x_va)[:,1]

metric_tr = accuracy_score(y_tr, np.where(y_tr_pred>=0.5, 1, 0))
metric_va = accuracy_score(y_va, np.where(y_va_pred>=0.5, 1, 0))

print("[accuracy] tr: {:.2f}, va: {:.2f}".format(metric_tr, metric_va))

#### スクリプト4-14: 説明変数の重要度の算出

In [ ]:
imp = pd.DataFrame({"col":x_train.columns, "imp":model.feature_importances_})
imp.sort_values("imp", ascending=False, ignore_index=True)

## 4.7.2 クロスバリデーションの場合
#### スクリプト4-15: モデル学習の実行（クロスバリデーションの場合）

In [ ]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary', 
    'metric': 'auc',
    'learning_rate': 0.1,
    'num_leaves': 16,
    'n_estimators': 100000,
    "random_state": 123,
    "importance_type": "gain",
}
    
metrics = []
imp = pd.DataFrame()

n_splits = 5
cv = list(StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=123).split(x_train, y_train))

for nfold in np.arange(n_splits):
    print("-"*20, nfold, "-"*20)
    idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
    x_tr, y_tr = x_train.loc[idx_tr, :], y_train.loc[idx_tr, :]
    x_va, y_va = x_train.loc[idx_va, :], y_train.loc[idx_va, :]
    print(x_tr.shape, y_tr.shape)
    print(x_va.shape, y_va.shape)
    print("y_train:{:.3f}, y_tr:{:.3f}, y_va:{:.3f}".format(
        y_train["Survived"].mean(),
        y_tr["Survived"].mean(),
        y_va["Survived"].mean(),
    ))
    
    model = lgb.LGBMClassifier(**params)
    model.fit(x_tr,
              y_tr,
              eval_set=[(x_tr,y_tr), (x_va,y_va)],
              early_stopping_rounds=100,
              verbose=100,
             )
    
#     # 2024/02/14環境で動かしたい場合はこのコードを利用してください。
#     model.fit(x_tr,
#               y_tr,
#               eval_set=[(x_tr,y_tr), (x_va,y_va)],
#               callbacks=[
#                   lgb.early_stopping(stopping_rounds=100, verbose=True),
#                   lgb.log_evaluation(100),
#               ],
#              )    

    y_tr_pred = model.predict(x_tr)
    y_va_pred = model.predict(x_va)
    metric_tr = accuracy_score(y_tr, y_tr_pred)
    metric_va = accuracy_score(y_va, y_va_pred)
    print("[accuracy] tr: {:.2f}, va: {:.2f}".format(metric_tr, metric_va))    
    metrics.append([nfold, metric_tr, metric_va])
    
    _imp = pd.DataFrame({"col":x_train.columns, "imp":model.feature_importances_, "nfold":nfold})
    imp = pd.concat([imp, _imp], axis=0, ignore_index=True)

print("-"*20, "result", "-"*20)
metrics = np.array(metrics)
print(metrics)

print("[cv ] tr: {:.2f}+-{:.2f}, va: {:.2f}+-{:.2f}".format(
    metrics[:,1].mean(), metrics[:,1].std(),
    metrics[:,2].mean(), metrics[:,2].std(),
))

imp = imp.groupby("col")["imp"].agg(["mean", "std"])
imp.columns = ["imp", "imp_std"]
imp = imp.reset_index(drop=False)

print("Done.")

#### スクリプト4-16: 説明変数の重要度の算出

In [ ]:
imp.sort_values("imp", ascending=False, ignore_index=True)

## 4.7.3 ベースラインの評価
#### スクリプト4-17: ベースライン検証用データの作成

In [ ]:
x_tr, x_va2, y_tr, y_va2 = train_test_split(x_train,
                                            y_train,
                                            test_size=0.2,
                                            shuffle=True,
                                            stratify=y_train,
                                            random_state=123)
print(x_tr.shape, y_tr.shape)
print(x_va2.shape, y_va2.shape)

#### スクリプト4-18: 学習データと検証データの分割（ホールドアウト検証）

In [ ]:
x_tr1, x_va1, y_tr1, y_va1 = train_test_split(x_tr,
                                              y_tr,
                                              test_size=0.2,
                                              shuffle=True,
                                              stratify=y_tr,
                                              random_state=789)
print(x_tr1.shape, y_tr1.shape)
print(x_va1.shape, y_va1.shape)

#### スクリプト4-19: モデル学習（ホールドアウト検証）

In [ ]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary', 
    'metric': 'auc',
    'learning_rate': 0.1,
    'num_leaves': 16,
    'n_estimators': 100000,
    "random_state": 123,
    "importance_type": "gain",
}
model = lgb.LGBMClassifier(**params)
model.fit(x_tr1,
          y_tr1,
          eval_set=[(x_tr1,y_tr1), (x_va1,y_va1)],
          early_stopping_rounds=100,
          verbose=10,
         )

# # 2024/02/14環境で動かしたい場合はこのコードを利用してください。
# model.fit(x_tr1,
#           y_tr1,
#           eval_set=[(x_tr1,y_tr1), (x_va1,y_va1)],
#           callbacks=[
#               lgb.early_stopping(stopping_rounds=100, verbose=True),
#               lgb.log_evaluation(10),
#           ],
#          )

#### スクリプト4-20: 検証データとベースライン検証用データの予測値算出

In [21]:
y_va1_pred = model.predict(x_va1)
y_va2_pred = model.predict(x_va2)

#### スクリプト4-21: モデル精度の比較

In [ ]:
print("[検証データ] acc: {:.4f}".format(accuracy_score(y_va1, y_va1_pred)))
print("[ベースライン検証用データ] acc: {:.4f}".format(accuracy_score(y_va2, y_va2_pred)))

#### スクリプト4-22: 誤差分布の比較

In [ ]:
print("検証データ")
print(confusion_matrix(y_va1, y_va1_pred))
print(confusion_matrix(y_va1, y_va1_pred, normalize="all"))
print("ベースライン検証用データ")
print(confusion_matrix(y_va2, y_va2_pred))
print(confusion_matrix(y_va2, y_va2_pred, normalize="all"))

#### スクリプト4-23: 予測値の分布比較

In [ ]:
# 予測値の確率値算出
y_va1_pred_prob = model.predict_proba(x_va1)[:,1]
y_va2_pred_prob = model.predict_proba(x_va2)[:,1]

# 確率値をヒストグラムで可視化
fig = plt.figure(figsize=(10,8))
# 検証データ
fig.add_subplot(2,1,1)
plt.title("検証データ")
plt.hist(y_va1_pred_prob[np.array(y_va1).reshape(-1)==1], bins=10, alpha=0.5, label="1")
plt.hist(y_va1_pred_prob[np.array(y_va1).reshape(-1)==0], bins=10, alpha=0.5, label="0")
plt.grid()
plt.legend()
# ベースライン検証用データ
fig.add_subplot(2,1,2)
plt.title("ベースライン検証用データ")
plt.hist(y_va2_pred_prob[np.array(y_va2).reshape(-1)==1], bins=10, alpha=0.5, label="1")
plt.hist(y_va2_pred_prob[np.array(y_va2).reshape(-1)==0], bins=10, alpha=0.5, label="0")
plt.grid()
plt.legend()

# 4.8 モデル推論
## 4.8.1 推論データセット作成
#### スクリプト4-24: 推論用データセットの作成

In [25]:
df_test = pd.read_csv("../input/titanic/test.csv")
x_test = df_test[["Pclass", "Fare"]]
id_test = df_test[["PassengerId"]]

## 4.8.2 学習済モデルを用いた推論
#### スクリプト4-25: 学習モデルによる推論

In [26]:
y_test_pred = model.predict(x_test)

#### スクリプト4-26: 提出用ファイルの作成

In [ ]:
df_submit = pd.DataFrame({"PassengerId":id_test["PassengerId"], "Survived":y_test_pred})
display(df_submit.head(5))
df_submit.to_csv("submission_baseline.csv", index=None)